In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', 1000)

# AboutYou Dataset

## Attributes Analysis
Loads the data.csv file and prints the most common attributes (tags). Then creates columns for the selected attributes and converts them to dummy variables.

In [3]:
data_path = '../../../data/zalando'
df = pd.read_csv(os.path.join(data_path, 'data.csv'), sep=';', encoding='utf-8')

In [4]:
df.describe()

,img_path,attributes,brand,category,color,id,img_url,model_img_urls,name
count,43413,43413,43413,43413,43413,43413,43413,43152,43413
unique,43413,42623,641,8,9,42622,42995,42735,20869
top,blusen-tuniken/AR321E023-K11.jpg,"Material & Pflegehinweise, Material Oberstoff: 99% Viskose, 1% Elasthan, Futter: 97% Polyester, 3% Elasthan, Materialkonstruktion: Spitze, Pflegehinweise: Chemische Reinigung, Mehr zu diesem Produkt, Stark transparente Partien, Rock und Top ohne Spitzenoberteil tragbar, Material Oberstoff Spitzenoberteil: 100 % Polyester, Ausschnitt: Rundhals, Verschluss: Reißverschluss, Muster: Unifarben, Details: Brustabnäher, Unterkleid, Artikelnummer: IV521C00D-A11, Modelgröße: Unser Model ist 179 cm groß und trägt Größe 36, Passform: Normal, Länge: Lang, Ärmellänge: Halbarm, Rückenbreite: 36 cm bei Größe 36, Gesamtlänge: 109 cm bei Größe 36, Kostenloser Versand & Rückversand, 100 Tage Rückgaberecht",Naketano,pullover-und-strickjacken,black,TRF21C01O-J11,https://mosaic04.ztat.net/vgs/media/packshot/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@2.jpg,"https://mosaic03.ztat.net/vgs/media/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@12.jpg, https://mosaic01.ztat.net/vgs/comet/IV/52/1C/00/DA/11/VIDEO/PREVIEW_IMG/1521623452509.jpg, https://mosaic04.ztat.net/vgs/media/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@13.jpg, https://mosaic03.ztat.net/vgs/media/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@11.jpg, https://mosaic04.ztat.net/vgs/media/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@9.jpg, https://mosaic03.ztat.net/vgs/media/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@10.jpg, https://mosaic03.ztat.net/vgs/media/pdp-zoom/IV/52/1C/00/DA/11/IV521C00D-A11@8.jpg",Strickpullover
freq,1,3,2520,11465,11269,3,3,3,1446


In [5]:
df['tags'] = df['attributes'].apply(lambda x: x.lower().split(', '))

Get count for each of the tags, if it appears in the attributes list of any of the images.

In [6]:
tag_count = {}
def count_tags(tags):
    for tag in tags:
        t = tag.lower()
        if t in tag_count:
            tag_count[t] += 1
        else:
            tag_count[t] = 1
            
a = df['tags'].apply(count_tags)

Sort by the most used tags and print

In [7]:
tag_df = pd.Series(tag_count).reset_index().rename(columns={'index': 'tag', 0: 'count'})
num_imgs = df.shape[0]
tag_df['ratio'] = (pd.to_numeric(tag_df['count']) / num_imgs * 100).round(2)
tag_df = tag_df[tag_df['ratio'] > 1]
tag_df = tag_df.sort_values('ratio', ascending=False)

In [8]:
tag_df['tag'].unique().tolist()

['kostenloser versand & rückversand',
 'material & pflegehinweise',
 'mehr zu diesem produkt',
 '100 tage rückgaberecht',
 'pflegehinweise:\xa0nicht trockner geeignet',
 'maschinenwäsche bei 30°c',
 'passform:\xa0normal',
 'muster:\xa0unifarben',
 'schonwaschgang',
 'länge:\xa0normale länge',
 'ausschnitt:\xa0rundhals',
 'ärmellänge:\xa0langarm',
 'pflegehinweise:\xa0maschinenwäsche bei 40°c',
 'material oberstoff:\xa0100% baumwolle',
 'materialkonstruktion:\xa0jersey',
 'details:\xa0elastischer bund',
 'verschluss:\xa0verdeckter zip-fly',
 'nicht trockner geeignet',
 'leibhöhe:\xa0normal',
 'hosentaschen:\xa0gesäßtaschen',
 'ärmellänge:\xa0extralanger arm',
 'material oberstoff:\xa0100% polyester',
 'seitentaschen',
 'länge:\xa0lang',
 'leibhöhe:\xa0hoch',
 'materialkonstruktion:\xa0sweat',
 'muster:\xa0print',
 'länge:\xa0kurz',
 'ärmellänge:\xa0kurzarm',
 'verschluss:\xa0reißverschluss',
 'verschluss:\xa0knopf',
 'pflegehinweise:\xa0maschinenwäsche bei 30°c',
 '5% elasthan',
 'länge

## Select useful attributes
Find attributes that are used a lot and can be useful for the application.
Create a special column in the dataframe with each of the useful tags.

In [9]:
def create_tag_column(df, tag_list, column_name):
    """ 
        Create a new column in the given dataframe with the specified column name. 
        The column holds the tag if the tag is in the tag_list otherwise it holds np.nan.
    """
    
    df[column_name] = df['tags'].apply(lambda x: [s for s in x if s in tag_list])
    df[column_name] = [x[0].split(':')[-1].strip() if len(x) > 0 else np.nan for x in df[column_name]]
    
    return df

### Ärmellänge

In [10]:
tag_df[tag_df['tag'].str.startswith('ärmellänge')]

,tag,count,ratio
55926,ärmellänge: langarm,10556,24.32
55923,ärmellänge: extralanger arm,5352,12.33
55925,ärmellänge: kurzarm,4413,10.17
55932,ärmellänge: ärmellos,2774,6.39
55921,ärmellänge: dreiviertelarm,2547,5.87
55924,ärmellänge: halbarm,1420,3.27
55931,ärmellänge: spaghettiträger,805,1.85
55922,ärmellänge: extrakurzer arm,701,1.61
55733,ärmellänge: 65 cm bei größe s,550,1.27
55710,ärmellänge: 64 cm bei größe s,549,1.26


In [11]:
aermel_list = tag_df[(tag_df['tag'].str.startswith('ärmellänge')) & ~(tag_df['tag'].str.contains('größe'))]['tag'].tolist()
df = create_tag_column(df, aermel_list, 'ärmellänge')

In [12]:
df['ärmellänge'].unique()

array(['langarm', 'kurzarm', 'dreiviertelarm', nan, 'extralanger arm',
       'ärmellos', 'spaghettiträger', 'extrakurzer arm', 'halbarm'], dtype=object)

### Länge

In [13]:
tag_df[tag_df['tag'].str.startswith('länge')]

,tag,count,ratio
50171,länge: normale länge,15738,36.25
50169,länge: lang,4933,11.36
50168,länge: kurz,4519,10.41
50167,länge: knöchellang,4048,9.32
50166,länge: knielang,3649,8.41
50164,länge: extra kurz,3546,8.17
50133,länge: 7/8 länge,2301,5.30
50172,länge: oberschenkellang,1752,4.04
50173,länge: wadenlang,1246,2.87
50165,länge: extra lang,851,1.96


In [14]:
laenge_list = tag_df[tag_df['tag'].str.startswith('länge')]['tag'].tolist()
df = create_tag_column(df, laenge_list, 'länge')

In [15]:
df['länge'].unique()

array(['normale länge', 'kurz', 'lang', nan, 'extra lang', 'extra kurz',
       'oberschenkellang', '7/8 länge', 'knöchellang', 'knielang',
       'wadenlang'], dtype=object)

### Ausschnitt

In [16]:
tag_df[tag_df['tag'].str.contains('ausschnitt')]

,tag,count,ratio
43783,ausschnitt: rundhals,11664,26.87
43786,ausschnitt: tiefer v-ausschnitt,2513,5.79
43784,ausschnitt: rückenausschnitt,1018,2.34
43788,ausschnitt: v-ausschnitt,959,2.21
43777,ausschnitt: carmen,739,1.70
43776,ausschnitt: cache-coeur,705,1.62
43785,ausschnitt: tiefer rundhals,643,1.48
43787,ausschnitt: u-boot,515,1.19


In [17]:
ausschnitt_list = tag_df[tag_df['tag'].str.contains('ausschnitt')]['tag'].tolist()
df = create_tag_column(df, ausschnitt_list, 'ausschnitt')

In [18]:
df['ausschnitt'].unique()

array(['tiefer v-ausschnitt', nan, 'rundhals', 'v-ausschnitt',
       'rückenausschnitt', 'u-boot', 'cache-coeur', 'carmen',
       'tiefer rundhals'], dtype=object)

### Muster

In [19]:
tag_df[tag_df['tag'].str.contains('muster')]

,tag,count,ratio
52387,muster: unifarben,20024,46.12
52386,muster: print,4680,10.78
52381,muster: meliert,3860,8.89
52377,muster: geblümt,3467,7.99
52379,muster: gestreift,3022,6.96
52378,muster: gepunktet,792,1.82
52380,muster: kariert,574,1.32


In [20]:
muster_list = tag_df[tag_df['tag'].str.contains('muster')]['tag'].tolist()
df = create_tag_column(df, muster_list, 'muster')

In [21]:
df['muster'].unique()

array(['unifarben', 'geblümt', nan, 'print', 'gestreift', 'kariert',
       'gepunktet', 'meliert'], dtype=object)

### Passform

In [22]:
tag_df[(tag_df['tag'].str.contains('passform'))]

,tag,count,ratio
52602,passform: normal,20439,47.08
52610,passform: tailliert,3800,8.75
52613,passform: weit geschnitten,3288,7.57
52606,passform: schmal,2811,6.48
52611,passform: tapered leg,1849,4.26
52600,passform: körpernah,1690,3.89
52608,passform: slim fit,1404,3.23
52598,passform: flare,1164,2.68
52607,passform: skinny,1114,2.57
52609,passform: straight leg,1076,2.48


In [23]:
passform_list = tag_df[(tag_df['tag'].str.contains('passform'))]['tag'].tolist()
df = create_tag_column(df, passform_list, 'passform')

In [24]:
df['passform'].unique()

array(['weit geschnitten', 'normal', 'tailliert', nan, 'slim fit',
       'körpernah', 'schmal', 'oversized', 'loose fit', 'straight leg',
       'flare', 'tapered leg', 'skinny'], dtype=object)

In [25]:
# df = df.drop(['attributes', 'tags', 'name', 'brand', 'id', 'model_img_urls'], axis=1)
df.head()

,img_path,attributes,brand,category,color,id,img_url,model_img_urls,name,tags,ärmellänge,länge,ausschnitt,muster,passform
0,blusen-tuniken/0VB21E00B-A11.jpg,"Material & Pflegehinweise, Material Oberstoff: 84% Baumwolle, 8% sonstige Materialien, 8% Leinen, Pflegehinweise: Handwäsche, chemische Reinigung möglich, Mehr zu diesem Produkt, Ärmel mit Knopfverschluss, Schnürung am Ausschnitt, Ausschnitt: Tiefer V-Ausschnitt, Transparenz: Stark, Muster: Unifarben, Artikelnummer: 0VB21E00B-A11, Modelgröße: Unser Model ist 176 cm groß und trägt Größe 36, Passform: Weit geschnitten, Länge: Normale Länge, Ärmellänge: Langarm, Ärmellänge: 58 cm bei Größe 36, Rückenbreite: 40 cm bei Größe 36, Gesamtlänge: 61 cm bei Größe 36, Kostenloser Versand & Rückversand, 100 Tage Rückgaberecht",Vanessa Bruno,blusen-tuniken,white,0VB21E00B-A11,https://mosaic02.ztat.net/vgs/media/packshot/pdp-zoom/0V/B2/1E/00/BA/11/0VB21E00B-A11@10.jpg,"https://mosaic01.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/BA/11/0VB21E00B-A11@8.jpg, https://mosaic02.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/BA/11/0VB21E00B-A11@9.jpg, https://mosaic01.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/BA/11/0VB21E00B-A11@7.jpg, https://mosaic01.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/BA/11/0VB21E00B-A11@6.jpg",IRA - Bluse,"[material & pflegehinweise, material oberstoff: 84% baumwolle, 8% sonstige materialien, 8% leinen, pflegehinweise: handwäsche, chemische reinigung möglich, mehr zu diesem produkt, ärmel mit knopfverschluss, schnürung am ausschnitt, ausschnitt: tiefer v-ausschnitt, transparenz: stark, muster: unifarben, artikelnummer: 0vb21e00b-a11, modelgröße: unser model ist 176 cm groß und trägt größe 36, passform: weit geschnitten, länge: normale länge, ärmellänge: langarm, ärmellänge: 58 cm bei größe 36, rückenbreite: 40 cm bei größe 36, gesamtlänge: 61 cm bei größe 36, kostenloser versand & rückversand, 100 tage rückgaberecht]",langarm,normale länge,tiefer v-ausschnitt,unifarben,weit geschnitten
1,blusen-tuniken/0VB21E00C-A11.jpg,"Material & Pflegehinweise, Material Oberstoff: 100% Baumwolle, Pflegehinweise: Nicht Trockner geeignet, Maschinenwäsche bei 30°C, Schonwaschgang, Mehr zu diesem Produkt, Stark transparente Partien, Ausschnitt: Karree, Muster: Unifarben, Artikelnummer: 0VB21E00C-A11, Modelgröße: Unser Model ist 180 cm groß und trägt Größe S, Passform: Normal, Länge: Normale Länge, Ärmellänge: Kurzarm, Rückenbreite: 39 cm bei Größe S, Gesamtlänge: 53 cm bei Größe S, Kostenloser Versand & Rückversand, 100 Tage Rückgaberecht",Vanessa Bruno,blusen-tuniken,white,0VB21E00C-A11,https://mosaic04.ztat.net/vgs/media/packshot/pdp-zoom/0V/B2/1E/00/CA/11/0VB21E00C-A11@10.jpg,"https://mosaic03.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/CA/11/0VB21E00C-A11@8.jpg, https://mosaic04.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/CA/11/0VB21E00C-A11@9.jpg, https://mosaic03.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/CA/11/0VB21E00C-A11@7.jpg, https://mosaic03.ztat.net/vgs/media/pdp-zoom/0V/B2/1E/00/CA/11/0VB21E00C-A11@6.jpg",ILIANE - Bluse,"[material & pflegehinweise, material oberstoff: 100% baumwolle, pflegehinweise: nicht trockner geeignet, maschinenwäsche bei 30°c, schonwaschgang, mehr zu diesem produkt, stark transparente partien, ausschnitt: karree, muster: unifarben, artikelnummer: 0vb21e00c-a11, modelgröße: unser model ist 180 cm groß und trägt größe s, passform: normal, länge: normale länge, ärmellänge: kurzarm, rückenbreite: 39 cm bei größe s, gesamtlänge: 53 cm bei größe s, kostenloser versand & rückversand, 100 tage rückgaberecht]",kurzarm,normale länge,NaN,unifarben,normal
2,blusen-tuniken/0VB21E00E-G11.jpg,"Material & Pflegehinweise, Material Oberstoff: 100% Seide, Pflegehinweise: Handwäsche, chemische Reinigung möglich, Mehr zu diesem Produkt, Schnürung am Ausschnitt, Ausschnitt: Tiefer V-Ausschnitt, Muster: Geblümt, Artikelnummer: 0VB21E00E-G11, Modelgröße: Unser Model ist 179 cm groß und trägt Größe 36, Passform: Weit geschnitten, Länge: Normale Länge, Ärmellänge: Dreiviertelarm, Ärmellänge: 50 cm 

## Create dummies
Turn the created attribute columns into dummy variables.

In [26]:
df = df.set_index('img_url')
df_dum = pd.get_dummies(df[['category', 'color', 'ärmellänge', 'länge', 'muster', 'passform', 'ausschnitt']]).reset_index()

In [27]:
print(df_dum.columns)
df_dum.head()

Index(['img_url', 'category_blusen-tuniken', 'category_hosen',
       'category_jacken-maentel', 'category_jeans', 'category_kleider',
       'category_pullover-und-strickjacken', 'category_roecke',
       'category_shirts', 'color_beige', 'color_black', 'color_blue',
       'color_gray', 'color_green', 'color_pink', 'color_red', 'color_white',
       'color_yellow', 'ärmellänge_dreiviertelarm',
       'ärmellänge_extrakurzer arm', 'ärmellänge_extralanger arm',
       'ärmellänge_halbarm', 'ärmellänge_kurzarm', 'ärmellänge_langarm',
       'ärmellänge_spaghettiträger', 'ärmellänge_ärmellos', 'länge_7/8 länge',
       'länge_extra kurz', 'länge_extra lang', 'länge_knielang',
       'länge_knöchellang', 'länge_kurz', 'länge_lang', 'länge_normale länge',
       'länge_oberschenkellang', 'länge_wadenlang', 'muster_geblümt',
       'muster_gepunktet', 'muster_gestreift', 'muster_kariert',
       'muster_meliert', 'muster_print', 'muster_unifarben', 'passform_flare',
       'passform_körpern

,img_url,category_blusen-tuniken,category_hosen,category_jacken-maentel,category_jeans,category_kleider,category_pullover-und-strickjacken,category_roecke,category_shirts,color_beige,...,passform_tapered leg,passform_weit geschnitten,ausschnitt_cache-coeur,ausschnitt_carmen,ausschnitt_rundhals,ausschnitt_rückenausschnitt,ausschnitt_tiefer rundhals,ausschnitt_tiefer v-ausschnitt,ausschnitt_u-boot,ausschnitt_v-ausschnitt
0,https://mosaic02.ztat.net/vgs/media/packshot/pdp-zoom/0V/B2/1E/00/BA/11/0VB21E00B-A11@10.jpg,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,https://mosaic04.ztat.net/vgs/media/packshot/pdp-zoom/0V/B2/1E/00/CA/11/0VB21E00C-A11@10.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://mosaic04.ztat.net/vgs/media/packshot/pdp-zoom/0V/B2/1E/00/EG/11/0VB21E00E-G11@2.jpg,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,https://mosaic03.ztat.net/vgs/media/packshot/pdp-zoom/1D/R2/1E/00/LA/11/1DR21E00L-A11@12.1.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,https://mosaic01.ztat.net/vgs/media/packshot/pdp-zoom/1D/R2/1E/00/NA/11/1DR21E00N-A11@12.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Join the relative image path based on the img_url and drop the img_url. This creates a dataframe that holds the relative image path and all its dummy variables.

In [28]:
df_dum = df_dum.groupby(['img_url']).max().reset_index()
df_paths = df.groupby(['img_url']).first()['img_path'].reset_index()
df_dum = df_paths.merge(df_dum, how='inner').drop(['img_url'], axis=1)

In [29]:
df_dum.head()

,img_path,category_blusen-tuniken,category_hosen,category_jacken-maentel,category_jeans,category_kleider,category_pullover-und-strickjacken,category_roecke,category_shirts,color_beige,...,passform_tapered leg,passform_weit geschnitten,ausschnitt_cache-coeur,ausschnitt_carmen,ausschnitt_rundhals,ausschnitt_rückenausschnitt,ausschnitt_tiefer rundhals,ausschnitt_tiefer v-ausschnitt,ausschnitt_u-boot,ausschnitt_v-ausschnitt
0,pullover-und-strickjacken/0RA41G002-C11.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hosen/0VB21A009-B11.jpg,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,roecke/0VB21B007-K11.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,jacken-maentel/0VB21P000-G11.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,hosen/10K21A001-Q11.jpg,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Save the file as csv

In [30]:
df_dum.to_csv('/Users/sonynka/HTW/IC/data/zalando/img_attr.csv', sep='\t', encoding='utf-8', index=False)